# Day 8: Demand II - Household Budget Survey

We will start by looking at a publicly available survey measuring household expenditures on energy components.

With that, we can forecast the effect of the energy crisis on household energy bills

The exercise is based on the post "Los efectos de la crisis energética en el presupuesto de los hogares: un impacto desigual":

https://nadaesgratis.es/admin/los-efectos-de-la-crisis-energetica-en-el-presupuesto-de-los-hogares-un-impacto-desigual



We first load relevant libraries

In [1]:

#using Pkg
#Pkg.update(["DataFrames", "CSV", "Plots", "Printf","StatsPlots","Statistics"])



In [2]:

    using DataFrames
    using CSV
    using Plots
    using Printf
    using StatsPlots
    using Binscatters


## Household expenditure surveys

Many countries survey households to understand the well-being of households across demographics.

These household surveys are used throughout the EU to construct energy-poverty measures. See a great resource here:

https://energy-poverty.ec.europa.eu/observing-energy-poverty/national-indicators_en

The data we will use today is based on household survey expenditures from Spain. We will be using the "micro" data, which contains the individual responses for each household. The data can be obtained here: 

https://www.ine.es/dyngs/INEbase/en/operacion.htm?c=Estadistica_C&cid=1254736176806&menu=resultados&idp=1254735976608

### Overview of dataset

We will work with a reduced version of the household expenditure survey. In the data, each row is a representative household with a given weight (factor) in the population at a given quarter

Variables from the survey at yearly level:
- decile: decile in the income distribution in which the household belongs to
- income: household income
- GASTO: = expenditure
- gasto_income_epf = GASTO / income
- CANTIDAD_ = quantity
- id_tur: whether the household has a regulated gas contract (cheaper)
- iprem: income criteria to be eligible for the "bono social"
- severo / vulnerable: whether the household is eligible to other subsidies, severo >> vulnerable
- bono: whether the household actually received the "bono" - a subsidy on the household electricity bill. We estimated that only 25% of those eligibles actually received it.
- See conditions for receiving the  subsidy: https://www.bonosocial.gob.es/#requisitos
- Among those, `income`*12 should not exceed `iprem`*1.5
 

*Price variables*:
1. Compute the average price from annual survey data = GASTO / CANTIDAD
2. Apply quaterly changes from real price data. This creates elec_price, gas_price and fuel_price

We repeat this procedure for electricity demand (elec_demand). This is, obtain quaterly changes from total electricity demand and impute it into CANTIDAD.

Real price and demand data comes from the Spanish market operator https://www.omie.es/

*Policies*:
- June 2021: IVA from 21% to 10%. With that we get `elec_price_no_iva`
- November 2021: increase "bono social" discount. Subsidies for vulnerable households went up from 25% to 60% of the total electricity bill, while severo increased from 40% to 70%. We apply these extra discounts to the variable ``elec_price_no_iva``. With that, we have the counterfactual prices without the policy ``elec_price_no_policy``
- April 2022: 20 cents fuel subsidy 


### Energy to income measures

We read the data and add the share of income used for energy expenditures.

Energy, as a basic good, represents a much higher level of expenditures for low-income households.

In [3]:


    df_hh = CSV.read("data_hh.csv", DataFrame)

    #Outcome variables depending on policy
    df_hh.energy_to_income = (df_hh.fuel_price .* df_hh.CANTIDAD_fuel .+
                              (df_hh.gas_price .*(1 .- df_hh.id_tur) .+ df_hh.tur_price .*df_hh.id_tur  ).*  df_hh.CANTIDAD_fuel .+
                              df_hh.elec_price_no_policy .* df_hh.elec_dmnd) ./ df_hh.income .*100;

    df_hh.energy_to_income_policy = (df_hh.fuel_price_policy .* df_hh.CANTIDAD_fuel .+
                              (df_hh.gas_price .*(1 .-df_hh.id_tur) .+ df_hh.tur_price .*df_hh.id_tur  ).*  df_hh.CANTIDAD_fuel .+
                              df_hh.elec_price .* df_hh.elec_dmnd) ./ df_hh.income .*100;

    df_hh


,quarter,decile,id,year,factor,income,GASTO_gas,CANTIDAD_gas,GASTO_electr
,String7,String3,Int64,Int64,Float64,Int64,Float64,Float64,Float64
1,2019 Q1,D1,63953,2019,332.468,558,72.9943,64.8,64.3157
2,2019 Q1,D1,63984,2019,988.648,773,0.0,0.0,45.0
3,2019 Q1,D1,64033,2019,425.276,666,0.0,0.0,40.0
4,2019 Q1,D1,64034,2019,451.393,460,0.0,0.0,75.0
5,2019 Q1,D1,64037,2019,1381.54,300,0.0,0.0,17.88
6,2019 Q1,D1,64038,2019,732.724,356,0.0,0.0,49.0
7,2019 Q1,D1,64041,2019,684.441,371,0.0,0.0,0.0
8,2019 Q1,D1,64058,2019,248.676,338,0.0,0.0,38.0
9,2019 Q1,D1,64091,2019,372.777,780,0.0,0.0,50.0


Rich households tend to consume more energy.

In [ ]:
# We do a weighted plot of electricity expenditures by monthly income
# Factor = survey weights to ensure sample is representative
binscatter(df_hh, @formula(GASTO_electr ~ income), weights=:factor) 
    


However, low-income houeholds spend a higher share and income.

In [ ]:
 
# We do a weighted plot of income and share of expenditures
# Factor = survey weights to ensure sample is representative
binscatter(df_hh, @formula(energy_to_income ~ income), weights=:factor) 


We can also summarize these measures by decile. We can also examine how the burden is changing over the years.

Note: We will also examine how relieve policies have impacted households' budgets.

In [ ]:

    
## Plot: Energy bill as a proportion of income ##
# Aggregate at quarter level, weighted by survey factors
plot_hh = combine(groupby(df_hh,[:quarter,:decile]), 
                [:energy_to_income,:energy_to_income_policy, :factor] => ((i, i_p, w) ->
                    (energy_to_income = (sum(i.* w) / sum(w)),
                     energy_to_income_policy = (sum(i_p .* w) / sum(w)))) => [:energy_to_income, :energy_to_income_policy]
                )
    
# selecting deciles
filter!(row ->(row.decile == "D1") | (row.decile == "D5") | (row.decile == "D10") ,plot_hh)

# plot
    plot(plot_hh.quarter, plot_hh.energy_to_income, group = plot_hh.decile,
        title = "Energy bill as a proportion of income",
        xlabel = "quarter",
        legend = :outerright,
        color=[:orange :black :red3])
    plot!(plot_hh.quarter, plot_hh.energy_to_income_policy, group = plot_hh.decile,
        color=[:orange :black :red3],
        linestyle=:dash) 


### Over-burdened households

We identify the households for which energy expenditures are a very large fraction of their expenses relative to their income.

This can help understand how the presence of mitigating policies affected the most vulnerable consumers.

In [ ]:
function household_prop(data::DataFrame; threshold=30.0, policy = 1)
    
    ## Plot: Households with energy bill above a threshold ##

    if policy == 1
        data.threshold_prop = ifelse.( data.energy_to_income_policy .>= threshold, 1, 0)
    else 
        data.threshold_prop = ifelse.( data.energy_to_income .>= threshold, 1, 0)
    end
                            
    plot_hh = combine(groupby(df_hh,[:year,:decile]), 
                [:threshold_prop, :factor] => ((th,  w) ->
                    (energy_to_income = (sum(th.* w) / sum(w)).*100,)) => [:threshold_prop]
                )

    plot_hh.dec = SubString.(plot_hh.decile, 2)
    plot_hh.dec = parse.(Float64,plot_hh.dec) 
    sort!(plot_hh,:dec)

    plot_prop = groupedbar(plot_hh.dec,plot_hh.threshold_prop, group = plot_hh.year,
               bar_position = :dodge, bar_width=0.7,
               xticks = (1:10, string.(1:10)),
               title = @sprintf("Households with energy bill above %2.1f percent", threshold),
               xlabel = "Income Decile"   ,
               legend = :outerright
              ,color=[:orange :black :red3 :skyblue1] 
              )

    return plot_prop
    
end

In [ ]:
household_prop(df_hh,threshold=50,policy=0)

## Follow-up exercises

1. Not always those households that need the most assistance are those that are in highest need. Make use of the variables `iprem`, `severo`, `vulnerable`, `bono` to examine this issue.
